In [ ]:
import subprocess  # For running the model
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
from ribasim import Model, Node
from ribasim.nodes import basin, flow_boundary, tabulated_rating_curve

---
title: "Quick start guide"
---

![](https://s3.deltares.nl/ribasim/doc-image/quickstart/cover.png){fig-align="left"}

# Introduction
Welcome to Ribasim!
This tutorial will help you get started with the basics of using Ribasim for river basin simulation.
In this tutorial, the schematization of models is done in Python using the Ribasim Python package.
The Ribasim Python package (named `ribasim`) simplifies the process of building, updating, and analyzing Ribasim model programmatically.
It also allows for the creation of entire models from base data, ensuring that your model setup is fully reproducible.

## Learning objectives
In this tutorial, we will focus on a fictional river basin called Crystal, which will serve as our case study.
The guide is divided into different modules, each covering various scenarios.
These include simulating natural flow, implementing reservoirs, and observing the impact of other structures.
While not all node types and possibilities will be demonstrated, the focus will be on the most commonly used and significant situations.
By the end of the tutorial, users will be able to:

- **Set up a basic Ribasim model**: Understand how to create a new model for a river basin using the Ribasim Python package.
- **Evaluate the impact of demands**: Introduce water demand (such as irrigation) and assess their effects on the river basin.
- **Modify and update models**: Learn how to update existing models with new data and changes.
- **Analyze simulation results**: Use built-in tools to analyze and interpret the results of your simulations.

## Prerequisites
First install the latest release of Ribasim as documented in [the installation guide](/install.qmd).

Download the `Crystal_Basin.zip` file from the website. Extract `Crystal_Basin.zip` and place it in the same directory as your Ribasim installation. This folder includes:

- `QuickStartGuide.pdf`
- `data`: Contains data inputs such as time series needed for running the case.
Additionally, your Python model (`.py`) and the results will also be saved in this folder.

# Crystal River Basin
We will examine a straightforward example of the Crystal river basin, which includes a main river and a single tributary flowing into the sea (see @fig-crystal-basin).
An average discharge of $44.45 \text{m}^3/\text{s}$ is measured at the confluence.
In this module, the basin is free of any activities, allowing the model to simulate the natural flow.
The next step is to include a demand (irrigation) that taps from a canal out of the main river.

![Crystal Basin based on natural flow](https://s3.deltares.nl/ribasim/doc-image/quickstart/Crystal-Basin-based-on-natural-flow.png){fig-align="left" #fig-crystal-basin}

After this module the user will be able to:

- Build a river basin model from scratch
- Understand the functionality of the Demand and Basin nodes
- Generate overview of results
- Evaluate the simulation results

## Natural flow

### Import packages
Before building the model we need to import some modules.
Open your favorite Python editor (Visual Studio Code, Jupyter, ...) and create a new script or notebook and name it `Crystal_1.1` and save it into your model folder `Crystal_Basin`.
Import the following modules in Python:

In [ ]:
import plotly.express as px
from ribasim.nodes import user_demand

### Setup paths and model configuration
Reference the paths of the Ribasim installation and model directory and define the time period (2022-01-01 until 2023-01-01) for the model simulation.
The coordinate reference system (CRS) is also required, and set to [EPSG:4326](https://epsg.io/4326), which means all coordinates are interpreted as latitude and longitude values.
The CRS is important for correctly placing Ribasim models on the map, but since this is a fictional model, it is not important.

In [ ]:
base_dir = Path("c:/bin/ribasim")
model_dir = base_dir / "Crystal_Basin"
data_path = model_dir / "data/input/ACTINFLW.csv"

starttime = "2022-01-01"
endtime = "2023-01-01"
model = Model(
    starttime=starttime,
    endtime=endtime,
    crs="EPSG:4326",
)

### FlowBoundary nodes
The Crystal basin consists of two inflow points, the tributary and the main Crystal river, we will call them Minor and Main respectively.
In order to define the time series flow rate ($\text{m}^3/\text{s}$) we read the discharge data from `ACTINFLW.csv`.
This is a monthly inflow timeseries from 2014 to 2023.
The used simulation period is defined by the `starttime` and `endtime` of the model, not by the input timeseries.

In [ ]:
pd.date_range(start="2022-01-01", end="2023-01-01", freq="MS")

In [ ]:
data = pd.DataFrame({
    "time": pd.date_range(start="2022-01-01", end="2023-01-01", freq="MS"),
    "main": [74.7, 57.9, 63.2, 183.9, 91.8, 47.5, 32.6, 27.6, 26.5, 25.1, 39.3, 37.8, 57.9],
    "minor": [16.3, 3.8, 3.0, 37.6, 18.2, 11.1, 12.9, 12.2, 11.2, 10.8, 15.1, 14.3, 11.8]
})  # fmt: skip
data["total"] = data["minor"] + data["main"]
display(data)

# Average and max inflow of the total inflow data timeseries
# From 2014 - 2023
print("Average inflow [m3/s]:", data["total"].mean())
print("Maximum inflow [m3/s]:", data["total"].max())

main = model.flow_boundary.add(
    Node(1, Point(0.0, 0.0), name="main"),
    [
        flow_boundary.Time(
            time=data.time,
            flow_rate=data.main,
        )
    ],
)

minor = model.flow_boundary.add(
    Node(2, Point(-3.0, 0.0), name="minor"),
    [
        flow_boundary.Time(
            time=data.time,
            flow_rate=data.minor,
        )
    ],
)

### Basin node (confluence)
To schematize the confluence from the tributary we will use the Basin node.
The node by itself portrays as water storage with a certain volume of water and can be used for different purposes, such as a reservoir, river reach, lake or in this case a confluence.
@fig-confluence visualizes a cross section of the confluence point in our model.

![Basin node concept for the confluence](https://s3.deltares.nl/ribasim/doc-image/quickstart/Basin-node-concept-for-the-confluence.png){fig-align="left" #fig-confluence}

@tbl-input1 shows the input data for the Basin node profile.

: Profile data for the basin node {#tbl-input1}

| Area [$\text{m}^2$] | Level [$\text{m}$] |
|---------------------|--------------------|
|    $672000.0$       |      $0.0$         |
|   $5600000.0$       |      $6.0$         |

Whilst in this case the level starts at $0.0$ and therefore happens to be the same as the depth, it should never be interpreted as a depth.
All water levels in Ribasim are assumed to be with respect to a shared reference datum, like mean sea level (MSL).
The first water level in the profile is the height of the Basin bottom above this reference datum.

To specify the Basin profile, the following code is used:

In [ ]:
confluence = model.basin.add(
    Node(3, Point(-1.5, -1), name="confluence"),
    [
        basin.Profile(area=[672000, 5600000], level=[0, 6]),
        basin.State(level=[4]),
        basin.Time(time=[starttime, endtime]),
    ],
)

### TabulatedRatingCurve
In the previous step we implemented a Basin node that functions as a confluence.
Conceptually, the Basin acts as a store of water, accumulating inflows and then releasing them.
A Basin cannot directly connect to another Basin, because the rules for water exchange between them need to be defined.
Connector nodes take care of this.
The first such node we introduce is the TabulatedRatingCurve.
It defines a relation between the water level ($h$) in the Basin and the outflow ($Q$) from the Basin.
This setup mimics the behavior of a gate or spillway, allowing us to model how varying water levels influence flow rates at the confluence.

As the two inflows come together at the confluence, we expect, as mentioned above, a discharge average of $44.45 \text{m}^3/\text{s}$.
It is therefore expected that the confluence Basin goes towards a level where the outflow is equal to the inflow via the rating curve.
Only then is the confluence Basin in equilibrium.
The maximum depth of the river is $6 \text{m}$, and the maximum inflow is $221.5 \text{m}^3/\text{s}$
The $Q(h)$ relationship in @tbl-input2 allows such inflows with reasonable water levels.

: Input data for the Tabulated Rating Curve {#tbl-input2}

| Water Level ($h$) [$\text{m}$] | Outflow ($Q$) [$\text{m}^3/\text{s}$] |
| -------------------------------|---------------------------------------|
| $0.0$                          | $0.0$                                 |
| $2.0$                          | $50.0$                                |
| $5.0$                          | $200.0$                               |

In Ribasim, the $Q(h)$ relation is a piecewise linear function, so the points in between will be linearly interpolated.
@fig-discharge illustrates the visual process and shows a progressive increase in discharge with rising water levels.
In this case this means:

- At level $0.0$: No discharge occurs. This represents a condition where the water level is too low for any flow to be discharged.
- At level $2.0$: Discharge is $50.0 \text{m}^3/\text{s}$. This is a bit above the average discharge rate, corresponding to the water level where normal flow conditions are established.
- At level $5.0$: Discharge rate reaches $200.0 \text{m}^3/\text{s}$. This discharge rate occurs at the water level during wet periods, indicating higher flow capacity.

![Discharge at corresponding water levels](https://s3.deltares.nl/ribasim/doc-image/quickstart/Discharge-at-corresponding-water-levels.png){fig-align="left" #fig-discharge}

Taking this into account, add the `TabulatedRatingCurve` as follows:

In [ ]:
weir = model.tabulated_rating_curve.add(
    Node(4, Point(-1.5, -1.5), name="weir"),
    [
        tabulated_rating_curve.Static(
            level=[0.0, 2, 5],
            flow_rate=[0.0, 50, 200],
        )
    ],
)

### Terminal node
Finally all the water will discharge into the sea.
We schematize this with the Terminal node, as it portrays the end point of the model, that can receive but not give water.
Besides the node number/name and location, no further input is needed.

In [ ]:
sea = model.terminal.add(Node(5, Point(-1.5, -3.0), name="sea"))

### Defining edges
Implement the connections (edges) between the nodes.

In [ ]:
model.edge.add(main, confluence)
model.edge.add(minor, confluence)
model.edge.add(confluence, weir)
model.edge.add(weir, sea)

### Visualization and model execution
Plot the schematization.

In [ ]:
model.plot();

Write the model configuration to the `TOML` file.
Name the output file `Crystal_1.1/ribasim.toml`:

In [ ]:
toml_path = model_dir / "Crystal_1.1/ribasim.toml"
model.write(toml_path)
cli_path = base_dir / "ribasim_windows/ribasim.exe"


The schematization should look like @fig-cs11.

![Schematization of the Crystal basin 1.1](https://s3.deltares.nl/ribasim/doc-image/quickstart/Schematization-of-the-Crystal-basin-1.1.png){fig-align="left" #fig-cs11}

After running `model.write` a subfolder `Crystal_1.1` is created, which contains the model input data and configuration:

- ribasim.toml: The model configuration
- database.gpkg: A GeoPackage containing the network geometry and input data of the nodes used.

Now run the model:

In [ ]:
result = subprocess.run([cli_path, toml_path], capture_output=True, encoding="utf-8")
print(result.stderr)
result.check_returncode()

### Post-processing results
Read the Arrow files and plot the simulated flows from different edges and the levels and storages at our confluence point:

In [ ]:
df_basin = pd.read_feather(model_dir / "Crystal_1.1/results/basin.arrow")

# Create pivot tables and plot for basin data
df_basin_wide = df_basin.pivot_table(
    index="time", columns="node_id", values=["storage", "level"]
)

# Skip the first timestep as it is the initialization step
df_basin_wide = df_basin_wide.iloc[1:]

# Plot level and storage on the same graph with dual y-axes
fig, ax1 = plt.subplots(figsize=(12, 6))

# Plot level on the primary y-axis
color = "b"
ax1.set_xlabel("Time")
ax1.set_ylabel("Level [m]", color=color)
ax1.plot(df_basin_wide.index, df_basin_wide["level"], color=color)
ax1.tick_params(axis="y", labelcolor=color)

# Create a secondary y-axis for storage
ax2 = ax1.twinx()
color = "r"
ax2.set_ylabel("Storage [m³]", color="r")
ax2.plot(df_basin_wide.index, df_basin_wide["storage"], linestyle="--", color=color)
ax2.tick_params(axis="y", labelcolor=color)

fig.tight_layout()  # Adjust layout to fit labels
plt.title("Basin Level and Storage Over Time")
plt.show()

In [ ]:
# Plot flow data
# Read the flow results
df_flow = pd.read_feather(model_dir / "Crystal_1.1/results/flow.arrow")
# Create 'edge' and 'flow_m3d' columns
df_flow["edge"] = list(zip(df_flow.from_node_id, df_flow.to_node_id))

# Create a pivot table
pivot_flow = df_flow.pivot_table(index="time", columns="edge", values="flow_rate")

# Skip the first timestep
pivot_flow = pivot_flow.iloc[1:]

line_styles = ["-", "--", "-", "-."]
num_styles = len(line_styles)

fig, ax = plt.subplots(figsize=(12, 6))
for i, column in enumerate(pivot_flow.columns):
    pivot_flow[column].plot(
        ax=ax, linestyle=line_styles[i % num_styles], linewidth=1.5, alpha=0.8
    )

# Set labels and title
ax.set_xlabel("Time")
ax.set_ylabel("Flow [m³/s]")
ax.legend(bbox_to_anchor=(1.15, 1), title="Edge")
plt.title("Flow Over Time")
plt.grid(True)
plt.show()

@fig-sim1 shows the storage and levels in the Basin node.

To accurately represent the relationship between water levels and discharge rates at this confluence, a TabulatedRatingCurve is used.
This setup mimics the behavior of a gate or spillway, allowing us to model how varying water levels influence flow rates at the confluence.
Since the basin node is functioning as a confluence rather than a storage reservoir, the simulated water levels and storage trends will closely follow the inflow patterns.
This is because there is no net change in storage; all incoming water is balanced by outgoing flow.

![Simulated basin level and storage](https://s3.deltares.nl/ribasim/doc-image/quickstart/Simulated-basin-level-and-storage.png){fig-align="left" #fig-sim1}

@fig-sim2 shows the discharges in $\text{m}^3/\text{s}$ on each edge.
Edge (3,4) represents the flow from the confluence to the tabulated rating curve and edge (4,5) represents the flow from the tabulated rating curve to the terminal.
Both show the same discharge over time.
Which is expected in a natural flow environment, as what is coming into the confluence must come out.

![Simulated flows on each edge](https://s3.deltares.nl/ribasim/doc-image/quickstart/Simulated-flows-on-each-edge.jpg){fig-align="left" #fig-sim2}

## Irrigation demand

Let us modify the environment to include agricultural activities within the basin, which necessitate irrigation.
Water is diverted from the main river through an irrigation canal, with a portion of it eventually returning to the main river (see @fig-irrigation).

![Crystal basin with irrigation](https://s3.deltares.nl/ribasim/doc-image/quickstart/Crystal-basin-with-irrigation.png){fig-align="left" #fig-irrigation}

For this schematization update, we need to incorporate three additional nodes:

- Basin: Represents a cross-sectional point where water is diverted.
- UserDemand: Represents the irrigation demand.
- TabulatedRatingCurve: Defines the remaining water flow from the main river at the diversion point.

### Add a second Basin node
Basin #3 will portray as the point in the river where the diversion takes place, getting the name `diversion`.
Its profile area at this intersection is slightly smaller than at the confluence.

In [ ]:
model.basin.add(
    Node(3, Point(-0.75, -0.5), name="diversion"),
    [
        basin.Profile(area=[500000, 5000000], level=[0, 6]),
        basin.State(level=[3]),
        basin.Time(time=[starttime, endtime]),
    ],
)

model.basin.add(
    Node(4, Point(-1.5, -1), name="confluence"),
    [
        basin.Profile(area=[672000, 5600000], level=[0, 6]),
        basin.State(level=[4]),
        basin.Time(time=[starttime, endtime]),
    ],
)

### Add the irrigation demand
A big farm company needs to apply irrigation to its field starting from April to September.
The irrigated field is $> 17000 \text{ ha}$ and requires around $5 \text{ mm/day}$.
In this case the farm company diverts from the main river an average flow rate of $10 \text{ m}^3/\text{s}$ and $12 \text{ m}^3/\text{s}$ during spring and summer, respectively.
Start of irrigation takes place on the 1st of April until the end of August.
The farmer taps water through a canal (demand).

For now, let's assume the return flow remains $0.0$ (`return_factor`).
Meaning all the supplied water to fulfill the demand is consumed and does not return back to the river.
The user demand node interpolates the demand values. Thus the following code needs to be implemented:

In [ ]:
model.user_demand.add(
    Node(6, Point(-1.5, 1.0), name="IrrA"),
    [
        user_demand.Time(
            demand=[0.0, 0.0, 10, 12, 12, 0.0],
            return_factor=0,
            min_level=0,
            priority=1,
            time=[
                starttime,
                "2022-03-31",
                "2022-04-01",
                "2022-07-01",
                "2022-09-30",
                "2022-10-01",
            ],
        )
    ],
)

### Add a TabulatedRatingCurve
The second TabulatedRatingCurve node will simulate the rest of the water that is left after diverting a part from the main river to the farm field.
The rest of the water will flow naturally towards the confluence:

In [ ]:
model.tabulated_rating_curve.add(
    Node(7, Point(-1.125, -0.75), name="MainDiv"),
    [
        tabulated_rating_curve.Static(
            level=[0.0, 1.5, 5],
            flow_rate=[0.0, 45, 200],
        )
    ],
)

It is up to the user to renumber the IDs of the nodes.
Applying the ID number based on the order of the nodes from up- to downstream keeps it more organized, but not necessary.

### Adjust the Terminal node ID and edges
Adjust the Terminal node ID.
Since we added more nodes we have more edges. Add and adjust the edges:

In [ ]:
model.terminal.add(Node(8, Point(-1.5, -3.0), name="Terminal"))

model.edge.add(model.flow_boundary[1], model.basin[3])
model.edge.add(model.flow_boundary[2], model.basin[4])
model.edge.add(model.basin[3], model.user_demand[6])
model.edge.add(model.user_demand[6], model.basin[4])
model.edge.add(model.basin[3], model.tabulated_rating_curve[7])
model.edge.add(model.tabulated_rating_curve[7], model.basin[4])
model.edge.add(model.basin[4], model.tabulated_rating_curve[5])
model.edge.add(model.tabulated_rating_curve[5], model.terminal[8])

### Plot model and run
Plot the schematization and run the model.
This time the new outputs should be written in a new folder called `Crystal_1.2`:

In [ ]:
model.plot()

toml_path = model_dir / "Crystal_1.2/ribasim.toml"
model.write(toml_path)
cli_path = base_dir / "ribasim_windows/ribasim.exe"

subprocess.run([cli_path, toml_path], check=True)

The schematization should look like @fig-cs12.

![Schematization of the Crystal basin with irrigation](https://s3.deltares.nl/ribasim/doc-image/quickstart/Schematization-of-the-Crystal-basin-with-irrigation.png){fig-align="left" #fig-cs12}

### Name the edges and basins
The names of each nodes are defined and saved in the geopackage.
However, in the dataframe this needs to be added by creating a dictionary and map it within the dataframe.

In [ ]:
# Dictionary mapping node_ids to names
edge_names = {
    (1, 3): "Main",
    (2, 4): "Minor",
    (3, 6): "IrrA Demand",
    (6, 4): "IrrA Drain",
    (3, 7): "Div2Main",
    (7, 4): "Main2Conf",
    (4, 5): "Conf2TRC",
    (5, 8): "TRC2Term",
}

# Dictionary mapping basins (node_ids) to names
node_names = {
    3: "Div",
    4: "Conf",
}

### Plot and compare the basin results
Plot the simulated levels and storages at the diverted section (basin 3) and at the confluence (basin 4).

In [ ]:
df_basin_div = df_basin_wide.xs("Div", axis=1, level=1, drop_level=False)
df_basin_conf = df_basin_wide.xs("Conf", axis=1, level=1, drop_level=False)


def plot_basin_data(
    ax, ax_twin, df_basin, level_color="b", storage_color="r", title="Basin"
):
    # Plot level data
    for idx, column in enumerate(df_basin["level"].columns):
        ax.plot(
            df_basin.index,
            df_basin["level"][column],
            linestyle="-",
            color=level_color,
            label=f"Level - {column}",
        )

    # Plot storage data
    for idx, column in enumerate(df_basin["storage"].columns):
        ax_twin.plot(
            df_basin.index,
            df_basin["storage"][column],
            linestyle="--",
            color=storage_color,
            label=f"Storage - {column}",
        )

    ax.set_ylabel("Level [m]", color=level_color)
    ax_twin.set_ylabel("Storage [m³]", color=storage_color)

    ax.tick_params(axis="y", labelcolor=level_color)
    ax_twin.tick_params(axis="y", labelcolor=storage_color)

    ax.set_title(title)

    # Combine legends from both axes
    lines, labels = ax.get_legend_handles_labels()
    lines_twin, labels_twin = ax_twin.get_legend_handles_labels()
    ax.legend(lines + lines_twin, labels + labels_twin, loc="upper left")


# Create subplots
fig, (ax1, ax3) = plt.subplots(2, 1, figsize=(12, 12), sharex=True)

# Plot Div basin data
ax2 = ax1.twinx()  # Secondary y-axis for storage
plot_basin_data(ax1, ax2, df_basin_div, title="Div Basin Level and Storage over Time")

# Plot Conf basin data
ax4 = ax3.twinx()  # Secondary y-axis for storage
plot_basin_data(ax3, ax4, df_basin_conf, title="Conf Basin Level and Storage over Time")

# Common X label
ax3.set_xlabel("Time")
fig.tight_layout()  # Adjust layout to fit labels
plt.show()

@fig-sim3 illustrates the water levels and storage capacities for each basin.
At the diverted section, where the profile is narrower than at the confluence, we anticipate lower storage and water levels compared to the confluence section.

When compared to the natural flow conditions, where no water is abstracted for irrigation (See Crystal 1.1), there is a noticeable decrease in both storage and water levels at the confluence downstream.
This reduction is attributed to the irrigation demand upstream with no return flow, which decreases the amount of available water in the main river, resulting in lower water levels at the confluence.

![Simulated basin levels and storages](https://s3.deltares.nl/ribasim/doc-image/quickstart/Simulated-basin-levels-and-storages.png){fig-align="left" #fig-sim3}

### Plot and compare the flow results
Plot the flow results in an interactive plotting tool.

In [ ]:
df_flow = pd.read_feather(model_dir / "Crystal_1.2/results/flow.arrow")
df_flow["edge"] = list(zip(df_flow.from_node_id, df_flow.to_node_id))
df_flow["name"] = df_flow["edge"].map(edge_names)

# Plot the flow data, interactive plot with Plotly
pivot_flow = df_flow.pivot_table(
    index="time", columns="name", values="flow_rate"
).reset_index()
fig = px.line(
    pivot_flow, x="time", y=pivot_flow.columns[1:], title="Flow Over Time [m3/s]"
)

fig.update_layout(legend_title_text="Edge")
fig.write_html(model_dir / "Crystal_1.2/plot_edges.html")
fig.show()

The plot will be saved as an HTML file, which can be viewed by dragging the file into an internet browser (@fig-sim4).

![Simulated flows of each edge](https://s3.deltares.nl/ribasim/doc-image/quickstart/Simulated-flows-of-each-edge.png){fig-align="left" #fig-sim4}

When selecting only the flow demanded by the User Demand node, or in other words the supply for irrigation increases at times when it is required (@fig-sim5) and the return flow remains zero, as the assumption defined before was that there is no drain.

![Supplied irrigation and return flow](https://s3.deltares.nl/ribasim/doc-image/quickstart/Supplied-irrigation-and-return-flow.png){fig-align="left" #fig-sim5}

@fig-sim6 shows the flow to the ocean (Terminal).
Compared to Crystal 1.1 the flow has decreased during the irrigated period.
Indicating the impact of irrigation without any drain.

![Simulated flow to Terminal](https://s3.deltares.nl/ribasim/doc-image/quickstart/Simulated-flow-to-Terminal.png){fig-align="left" #fig-sim6}

# Reservoirs and Public Water Supply
Due to the increase of population and climate change Crystal city has implemented a reservoir upstream to store water for domestic use (See @fig-reservoir).
The reservoir is to help ensure a reliable supply during dry periods.
In this module, the user will update the model to incorporate the reservoir's impact on the whole Crystal Basin.

![Crystal basin with demands and a reservoir](https://s3.deltares.nl/ribasim/doc-image/quickstart/Crystal-basin-with-demands-and-a-reservoir.png){fig-align="left" #fig-reservoir}

## Reservoir
### Add a Basin
This time Basin #3 will function as a reservoir instead of a diversion, meaning it's storage and levels will play an important role for the users (the city and the farmer).
The reservoir has a max. area of $32.3 \text{ km}^2$ and a max. depth of $7 \text{ m}$.
The profile of Basin #3 should change to:

In [ ]:
model.basin.add(
    Node(3, Point(-0.75, -0.5), name="Rsv"),
    [
        basin.Profile(area=[20000000, 32300000], level=[0, 7]),
        basin.State(level=[3.5]),
        basin.Time(time=[starttime, endtime]),
    ],
)

### Adjust the code
Adjust the naming of the Basin in the dictionary mapping and the saving file should be `Crystal_2.1` instead of `*_1.2`.

In [ ]:
toml_path = model_dir / "Crystal_2.1/ribasim.toml"
model.write(toml_path)
cli_path = base_dir / "ribasim_windows/ribasim.exe"

# Dictionary mapping node_ids to names
edge_names = {
    (1, 3): "Main",
    (2, 4): "Minor",
    (3, 6): "IrrA Demand",
    (6, 4): "IrrA Drain",
    (3, 7): "Rsv2Main",
    (7, 4): "Main2Conf",
    (4, 5): "Conf2TRC",
    (5, 8): "TRC2Term",
}

# Dictionary mapping basins (node_ids) to names
node_names = {
    3: "Rsv",
    4: "Conf",
}

df_basin = pd.read_feather(model_dir / "Crystal_2.1/results/basin.arrow")

# Create pivot tables and plot for basin data
df_basin_rsv = df_basin_wide.xs("Rsv", axis=1, level=1, drop_level=False)
df_basin_conf = df_basin_wide.xs("Conf", axis=1, level=1, drop_level=False)

# Plot Rsv basin data
ax2 = ax1.twinx()  # Secondary y-axis for storage
plot_basin_data(ax1, ax2, df_basin_rsv, title="Reservoir Level and Storage Over Time")

In [ ]:
# Sample data loading and preparation
df_flow = pd.read_feather(model_dir / "Crystal_2.1/results/flow.arrow")
df_flow["edge"] = list(zip(df_flow.from_node_id, df_flow.to_node_id))
df_flow["name"] = df_flow["edge"].map(edge_names)

# Plot the flow data, interactive plot with Plotly
pivot_flow = df_flow.pivot_table(
    index="time", columns="name", values="flow_rate"
).reset_index()
fig = px.line(
    pivot_flow, x="time", y=pivot_flow.columns[1:], title="Flow Over Time [m3/s]"
)

fig.update_layout(legend_title_text="Edge")
fig.write_html(model_dir / "Crystal_2.1/plot_edges.html")
fig.show()

### Plotting results
@fig-sim7 illustrates the new storage and water level at the reservoir.
As expected, after increasing the profile of basin 3 to mimic the reservoir, its storage capacity increased as well.

![Simulated basin storages and levels](https://s3.deltares.nl/ribasim/doc-image/quickstart/Simulated-basin-storages-and-levels.png){fig-align="left" #fig-sim7}

## Public Water Supply

### Rename the saving files
Rename the files to `Crystal_2.2`

### Add a demand node
$50.000$ people live in Crystal City.
To represents the total flow rate or abstraction rate required to meet the water demand of $50.000$ people, another demand node needs to be added assuming a return flow of $60%$.

In [ ]:
model.user_demand.add(
    Node(9, Point(0.0, -0.25), name="PWS"),
    [
        user_demand.Time(
            # Total demand in m³/s
            demand=[
                0.07,
                0.08,
                0.09,
                0.10,
                0.12,
                0.14,
                0.15,
                0.14,
                0.12,
                0.10,
                0.09,
                0.08,
            ],
            return_factor=0.6,
            min_level=0,
            priority=1,
            time=[
                starttime,
                "2022-02-01",
                "2022-03-01",
                "2022-04-01",
                "2022-05-01",
                "2022-06-01",
                "2022-07-01",
                "2022-08-01",
                "2022-09-01",
                "2022-10-01",
                "2022-11-01",
                "2022-12-01",
            ],
        )
    ],
)

### Add the edges
The connection between the reservoir and the demand node needs to be defined:

In [ ]:
model.edge.add(model.flow_boundary[1], model.basin[3])
model.edge.add(model.flow_boundary[2], model.basin[4])
model.edge.add(model.basin[3], model.user_demand[6])
model.edge.add(model.basin[3], model.user_demand[9])
model.edge.add(model.user_demand[6], model.basin[4])
model.edge.add(model.user_demand[9], model.basin[4])
model.edge.add(model.basin[3], model.tabulated_rating_curve[7])
model.edge.add(model.tabulated_rating_curve[7], model.basin[4])
model.edge.add(model.basin[4], model.tabulated_rating_curve[5])
model.edge.add(model.tabulated_rating_curve[5], model.terminal[8])

### Adjust the name dictionaries

In [ ]:
# Dictionary mapping node_ids to names
edge_names = {
    (1, 3): "Main",
    (2, 4): "Minor",
    (3, 6): "IrrA Demand",
    (6, 4): "IrrA Drain",
    (3, 9): "PWS Demand",
    (9, 4): "PWS Return",
    (3, 7): "Rsv2Main",
    (7, 4): "Main2Conf",
    (4, 5): "Conf2TRC",
    (5, 8): "TRC2Term",
}

### Check the simulated demands
@fig-sim8 shows the flow to (PWS Demand) and out (PWS Return) of the PWS node.
@fig-sim9 shows the downstream flow to the ocean.
The impact is clear.
Due to the demands upstream (irrigation and public water supply) an expected decrease of discharge is shown downstream.

![Simulated flows to and from the city](https://s3.deltares.nl/ribasim/doc-image/quickstart/Simulated-flows-to-and-from-the-city.png){fig-align="left" #fig-sim8}

![Simulated basin storages and levels](https://s3.deltares.nl/ribasim/doc-image/quickstart/Simulated-basin-storages-and-levels-reservoir.png){fig-align="left" #fig-sim9}